# Testing cross project artifacts

## 1. artifact generating function 

In [1]:
#mlrun: start-code

import mlrun
import pandas as pd 
import json
import os
from xgboost import XGBClassifier
import pickle
from mlrun.artifacts.base import DirArtifact
from mlrun import MLClientCtx
from sklearn.datasets import load_iris
from io import BytesIO
from sklearn.model_selection import train_test_split

def get_dataitem(context: MLClientCtx,
                             key: str):
    
    for artifact in context.artifacts:
        if artifact['kind'] == 'model' and artifact['metadata'].get('key',None) == key:
            return mlrun.get_dataitem(artifact['spec']['target_path'] + artifact['spec']['model_file'])
        elif artifact['kind'] == 'dataset' and artifact['metadata'].get('key',None) == key:
            return mlrun.get_dataitem(artifact['spec']['target_path'])
        elif artifact['metadata'].get('key',None) == key:
            return mlrun.get_dataitem(artifact['spec']['target_path'])
    context.logger.info('Artifact not found')
    
def log_transactions(context: MLClientCtx,
                    ):
        
    # uploading new artifact 
    df_encode = pd.DataFrame(load_iris()['data']).to_json().encode()
    context.log_artifact('encoded_iris-'+context.artifact_path[:2], body=df_encode, local_path='encoded_iris-'+context.artifact_path[:2]+'.csv')
    # reading artifact
    trans_df = pd.DataFrame(json.loads(get_dataitem(context, 'encoded_iris-'+context.artifact_path[:2]).get()))
    context.logger.info(f'dataframe shape : {trans_df.shape}')
    
    # training the model (for serving purposes )
    bst = XGBClassifier(n_estimators=2, max_depth=2, learning_rate=1, objective='binary:logistic')
    X,y = load_iris(return_X_y=True)
    X_train, x_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)
    bst.fit(X_train, y_train)
    # logging a model
    context.log_model('bst_model', body=pickle.dumps(bst), model_file='bst.pkl')
    # getting the model remote
    model = pickle.loads(get_dataitem(context, 'bst_model').get())
    context.logger.info(f'logged model : {model.__class__}')
    
    # Logging directory
    context.log_artifact(DirArtifact(key='my_project', target_path=context.artifact_path))
    
    # Logging dataset 
    context.log_dataset(key = 'iris_dataset-'+context.artifact_path[:2],
                        df = pd.DataFrame(json.loads(get_dataitem(context, 'encoded_iris-'+context.artifact_path[:2]).get())),
                        local_path='iris_dataset-'+context.artifact_path[:2]+'.csv')
    # Getting dataset
    context.logger.info(f'logged dataset {get_dataitem(context, "iris_dataset-" + context.artifact_path[:2]).as_df().shape}')
    
    return
            
#mlrun: end-code

## 2. Creating projects, setting & running artifact generating function

In [2]:
import mlrun
import os

# Initialize the MLRun project object
project1 = mlrun.get_or_create_project('testing-notebooks1',user_project=True,context=os.path.join(os.getcwd(), 'test-notebooks1'))

project1.set_function(name='log_transactions', kind='job', image='mlrun/ml-models', handler='log_transactions')

project1.get_function('log_transactions').run(local=False)

> 2022-12-21 07:35:32,523 [info] Created and saved project testing-notebooks1-dani: {'from_template': None, 'overwrite': False, 'context': '/User/test-notebooks/project_transfer/test-notebooks1', 'save': True}
> 2022-12-21 07:35:32,524 [info] created project testing-notebooks1 and saved in MLRun DB
> 2022-12-21 07:35:55,564 [info] starting run log-transactions-log_transactions uid=7fb2508bda86428ca3b218b64dc3d440 DB=http://mlrun-api:8080
> 2022-12-21 07:35:55,760 [info] Job is running in the background, pod: log-transactions-log-transactions-hfng6
> 2022-12-21 07:35:59,174 [info] dataframe shape : (150, 4)
> 2022-12-21 07:35:59,258 [info] logged model : <class 'xgboost.sklearn.XGBClassifier'>
> 2022-12-21 07:35:59,395 [info] logged dataset (150, 5)
> 2022-12-21 07:35:59,446 [info] To track results use the CLI: {'info_cmd': 'mlrun get run 7fb2508bda86428ca3b218b64dc3d440 -p testing-notebooks1-dani', 'logs_cmd': 'mlrun logs 7fb2508bda86428ca3b218b64dc3d440 -p testing-notebooks1-dani'}
> 

project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
testing-notebooks1-dani,...4dc3d440,0,Dec 21 07:35:58,completed,log-transactions-log_transactions,v3io_user=danikind=jobowner=danimlrun/client_version=1.2.1-rc4host=log-transactions-log-transactions-hfng6,,,,encoded_iris-v3bst_modelmy_projectiris_dataset-v3


> 2022-12-21 07:36:02,101 [info] run executed, status=completed


In [3]:
# Initialize the MLRun project object
project2 = mlrun.get_or_create_project('testing-notebooks2',user_project=True,context=os.path.join(os.getcwd(), 'test-notebooks2'))

project2.set_function(name='log_transactions', kind='job', image='mlrun/ml-models', handler='log_transactions')

project2.get_function('log_transactions').run(local=False)

> 2022-12-21 07:36:17,389 [info] Created and saved project testing-notebooks2-dani: {'from_template': None, 'overwrite': False, 'context': '/User/test-notebooks/project_transfer/test-notebooks2', 'save': True}
> 2022-12-21 07:36:17,390 [info] created project testing-notebooks2 and saved in MLRun DB
> 2022-12-21 07:36:40,966 [info] starting run log-transactions-log_transactions uid=5ee40c257c6b434a9770ecd52e27ed02 DB=http://mlrun-api:8080
> 2022-12-21 07:36:41,160 [info] Job is running in the background, pod: log-transactions-log-transactions-rpzgg
> 2022-12-21 07:36:44,548 [info] dataframe shape : (150, 4)
> 2022-12-21 07:36:44,640 [info] logged model : <class 'xgboost.sklearn.XGBClassifier'>
> 2022-12-21 07:36:44,773 [info] logged dataset (150, 5)
> 2022-12-21 07:36:44,828 [info] To track results use the CLI: {'info_cmd': 'mlrun get run 5ee40c257c6b434a9770ecd52e27ed02 -p testing-notebooks2-dani', 'logs_cmd': 'mlrun logs 5ee40c257c6b434a9770ecd52e27ed02 -p testing-notebooks2-dani'}
> 

project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
testing-notebooks2-dani,...2e27ed02,0,Dec 21 07:36:44,completed,log-transactions-log_transactions,v3io_user=danikind=jobowner=danimlrun/client_version=1.2.1-rc4host=log-transactions-log-transactions-rpzgg,,,,encoded_iris-v3bst_modelmy_projectiris_dataset-v3


> 2022-12-21 07:36:47,700 [info] run executed, status=completed


## 3. Importing/Exporting artifacts

### 3.1 base artifact

In [4]:
# Exporting project2 artifact
project2.get_artifact('log-transactions-log_transactions_encoded_iris-v3').export('proj2_encoded_iris.yaml')

# Importing the artifact from project1
project1.import_artifact('proj2_encoded_iris.yaml',
                         new_key = 'imported_proj1_encoded_iris')

# Testing the imported artifact
pd.DataFrame(json.loads(project1.get_artifact('imported_proj1_encoded_iris').to_dataitem().get())).head()

,0,1,2,3
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


### 3.2 dataset artifact

In [5]:
# Exporting project2 model artifact
project2.get_artifact('log-transactions-log_transactions_iris_dataset-v3').export('proj2_dataset.yaml')

# importing model artifact from project2
project1.import_artifact('proj2_dataset.yaml',
                         new_key = 'imported_proj1_dataset')

# Testing the imported dataset artifact
project1.get_artifact('imported_proj1_dataset').to_dataitem().as_df()

,Unnamed: 0,0,1,2,3
0,0,5.1,3.5,1.4,0.2
1,1,4.9,3.0,1.4,0.2
2,2,4.7,3.2,1.3,0.2
3,3,4.6,3.1,1.5,0.2
4,4,5.0,3.6,1.4,0.2
...,...,...,...,...,...
145,145,6.7,3.0,5.2,2.3
146,146,6.3,2.5,5.0,1.9
147,147,6.5,3.0,5.2,2.0
148,148,6.2,3.4,5.4,2.3


### 3.3 model artifact

In [6]:
# Exporting project1 model artifact
project1.get_artifact('log-transactions-log_transactions_bst_model').export('proj1_model.yaml')

# importing model artifact from project2
project2.import_artifact('proj1_model.yaml',
                         new_key = 'imported_proj2_model')


# Testing the imported model artifact
project2_model = pickle.loads(project2.get_artifact('imported_proj2_model')._get_file_body())
project2_model.predict(pd.DataFrame(json.loads((project1.get_artifact('log-transactions-log_transactions_encoded_iris-v3').to_dataitem().get())))[:5])

array([0, 0, 0, 0, 0])

### 3.4 dir artifact

In [7]:
try: 
    # Exporting project2 dir artifact
    project1.get_artifact('log-transactions-log_transactions_my_project').export('proj1_dir.yaml')

    # importing dir artifact from project2
    project2.import_artifact('proj1_dir.yaml',
                             new_key = 'imported_proj2_dir')

    # Testing the imported dir artifact
    project1.get_artifact('imported_proj2_dir').to_dataitem().listdir()
except Exception as e:
    print(e)

'LegacyDirArtifact' object has no attribute 'export'


## 4. Importing/Exporting functions

### 4.1 Serving function

#### 4.1.1 Creating serving function, adding model, predicting and deploying

In [8]:
# Setting project1 serving function
project1.set_function(name='proj1_serving',func = mlrun.new_function(name='proj1_serving', kind='serving',image='mlrun/ml-models', command = []))

# adding project2 model to project1 serving function
project1.get_function('proj1_serving').add_model(key = 'my_model',
                                                 class_name = "mlrun.frameworks.xgboost.XGBoostModelServer",
                                                 model_path = project2.get_artifact('log-transactions-log_transactions_bst_model').target_path)

In [9]:
# Testing the fused model serving function
server = project1.get_function('proj1_serving').to_mock_server()

server.test(body={'inputs': 
                  pd.DataFrame(json.loads(project2.get_artifact('log-transactions-log_transactions_encoded_iris-v3').to_dataitem().get())).values.tolist()[-5:]})

> 2022-12-21 07:36:48,583 [warning] run command, file or code were not specified
> 2022-12-21 07:36:53,381 [info] model my_model was loaded
> 2022-12-21 07:36:53,381 [info] Loaded ['my_model']


{'id': '5d1107e920e648b39f43264559a13a5f',
 'model_name': 'my_model',
 'outputs': [2, 2, 2, 2, 2]}

In [11]:
project1.deploy_function('proj1_serving')

> 2022-12-21 07:36:53,530 [info] Starting remote function deploy
2022-12-21 07:36:53  (info) Deploying function
2022-12-21 07:36:53  (info) Building
2022-12-21 07:36:53  (info) Staging files and preparing base images
2022-12-21 07:36:53  (info) Building processor image
2022-12-21 07:38:18  (info) Build complete
2022-12-21 07:38:24  (info) Function deploy complete
> 2022-12-21 07:38:25,216 [info] successfully deployed function: {'internal_invocation_urls': ['nuclio-testing-notebooks1-dani-proj1-serving.default-tenant.svc.cluster.local:8080'], 'external_invocation_urls': ['testing-notebooks1-dani-proj1-serving-testing-notebooks1-dani.default-tenant.app.jmglmvqnganv.iguazio-cd1.com/']}


DeployStatus(state=ready, outputs={'endpoint': 'http://testing-notebooks1-dani-proj1-serving-testing-notebooks1-dani.default-tenant.app.jmglmvqnganv.iguazio-cd1.com/', 'name': 'testing-notebooks1-dani-proj1-serving'})

In [12]:
import time
time.sleep(5)

project1.get_function('proj1_serving').invoke(
    path='/v2/models/my_model/infer', 
    body={'inputs': pd.DataFrame(json.loads(project2.get_artifact('log-transactions-log_transactions_encoded_iris-v3').to_dataitem().get())).values.tolist()[-5:]}
)

> 2022-12-21 07:38:30,404 [info] invoking function: {'method': 'POST', 'path': 'http://nuclio-testing-notebooks1-dani-proj1-serving.default-tenant.svc.cluster.local:8080/v2/models/my_model/infer'}


{'id': 'af2ea067-1e4b-4c64-8d19-c62b3848bf5b',
 'model_name': 'my_model',
 'outputs': [2, 2, 2, 2, 2]}

#### 4.1.2 Importing & Exporting already deployed serving function

In [13]:
# exporting project1 deployed serving function
project1.get_function('proj1_serving').export('proj1_serving.yaml')

# importing project1 serving function from project2
project2.set_function(name='proj2_imported_serving', func='../proj1_serving.yaml')
print(project2.get_function('proj2_imported_serving').is_deployed())

> 2022-12-21 07:38:30,502 [info] function spec saved to path: proj1_serving.yaml
True


In [14]:
try:
    # Testing the imported already deployed function
    server = project2.get_function('proj2_imported_serving').to_mock_server()

    server.test(body={'inputs': 
                      pd.DataFrame(json.loads(project2.get_artifact('log-transactions-log_transactions_encoded_iris-v3').to_dataitem().get())).values.tolist()[-5:]})
except Exception as e:
    print(e)

command file http://testing-notebooks1-dani-proj1-serving-testing-notebooks1-dani.default-tenant.app.jmglmvqnganv.iguazio-cd1.com/ not found


In [15]:
# Redeploying the already deployed function
project2.deploy_function('proj2_imported_serving')

> 2022-12-21 07:38:30,691 [info] Starting remote function deploy
2022-12-21 07:38:30  (info) Deploying function
2022-12-21 07:38:30  (info) Building
2022-12-21 07:38:30  (info) Staging files and preparing base images
2022-12-21 07:38:30  (info) Building processor image
2022-12-21 07:40:16  (info) Build complete
2022-12-21 07:40:24  (info) Function deploy complete
> 2022-12-21 07:40:24,266 [info] successfully deployed function: {'internal_invocation_urls': ['nuclio-testing-notebooks2-dani-proj1-serving.default-tenant.svc.cluster.local:8080'], 'external_invocation_urls': ['testing-notebooks2-dani-proj1-serving-testing-notebooks2-dani.default-tenant.app.jmglmvqnganv.iguazio-cd1.com/']}


DeployStatus(state=ready, outputs={'endpoint': 'http://testing-notebooks2-dani-proj1-serving-testing-notebooks2-dani.default-tenant.app.jmglmvqnganv.iguazio-cd1.com/', 'name': 'testing-notebooks2-dani-proj1-serving'})

##### Testing the redeployed imported-deployed-function

In [16]:
time.sleep(5) 
project2.get_function('proj2_imported_serving').invoke(
    path='/v2/models/my_model/infer', 
    body={'inputs': pd.DataFrame(json.loads(project2.get_artifact('log-transactions-log_transactions_encoded_iris-v3').to_dataitem().get())).values.tolist()[-5:]}
)

> 2022-12-21 07:40:29,580 [info] invoking function: {'method': 'POST', 'path': 'http://nuclio-testing-notebooks2-dani-proj1-serving.default-tenant.svc.cluster.local:8080/v2/models/my_model/infer'}


{'id': 'ce5370fb-a51e-42e3-a270-3f470a9098c5',
 'model_name': 'my_model',
 'outputs': [2, 2, 2, 2, 2]}

#### 4.1.3 Importing & Exporting undeployed serving function

In [17]:
# Setting project2 serving function
project2.set_function(mlrun.new_function(name='proj2_serving', kind='serving',image='mlrun/ml-models', command = []))

# adding project1 model to project2 serving function
project2.get_function('proj2-serving').add_model(key = 'my_model',
                                                 class_name = "mlrun.frameworks.xgboost.XGBoostModelServer",
                                                 model_path = project1.get_artifact('log-transactions-log_transactions_bst_model').target_path)

In [18]:
# exporting project2 not deployed serving function
project2.get_function('proj2-serving').export('proj2_serving.yaml')

# Importing project2 serving function from project1
project1.set_function(name='proj1_imported_serving', func='../proj2_serving.yaml')
print(project1.get_function('proj1_imported_serving').is_deployed())

> 2022-12-21 07:40:29,788 [info] function spec saved to path: proj2_serving.yaml
True


In [19]:
# Testing the imported undeployed function
server = project1.get_function('proj1_imported_serving').to_mock_server()

server.test(body={'inputs': 
                  pd.DataFrame(json.loads(project2.get_artifact('log-transactions-log_transactions_encoded_iris-v3').to_dataitem().get())).values.tolist()[-5:]})

> 2022-12-21 07:40:29,934 [warning] run command, file or code were not specified
> 2022-12-21 07:40:30,433 [info] model my_model was loaded
> 2022-12-21 07:40:30,433 [info] Loaded ['my_model']


{'id': '9b3c1b5db28540378711426457f3b175',
 'model_name': 'my_model',
 'outputs': [2, 2, 2, 2, 2]}

In [20]:
# Deploying the undeployed imported function
project1.deploy_function('proj1_imported_serving')

> 2022-12-21 07:40:30,561 [info] Starting remote function deploy
2022-12-21 07:40:30  (info) Deploying function
2022-12-21 07:40:30  (info) Building
2022-12-21 07:40:30  (info) Staging files and preparing base images
2022-12-21 07:40:30  (info) Building processor image
2022-12-21 07:41:45  (info) Build complete
2022-12-21 07:42:24  (info) Function deploy complete
> 2022-12-21 07:42:24,386 [info] successfully deployed function: {'internal_invocation_urls': ['nuclio-testing-notebooks1-dani-proj2-serving.default-tenant.svc.cluster.local:8080'], 'external_invocation_urls': ['testing-notebooks1-dani-proj2-serving-testing-notebooks1-dani.default-tenant.app.jmglmvqnganv.iguazio-cd1.com/']}


DeployStatus(state=ready, outputs={'endpoint': 'http://testing-notebooks1-dani-proj2-serving-testing-notebooks1-dani.default-tenant.app.jmglmvqnganv.iguazio-cd1.com/', 'name': 'testing-notebooks1-dani-proj2-serving'})

In [21]:
time.sleep(5)

# Testing the imported then deployed function
project1.get_function('proj1_imported_serving').invoke(
    path='/v2/models/my_model/infer', 
    body={'inputs': pd.DataFrame(json.loads(project2.get_artifact('log-transactions-log_transactions_encoded_iris-v3').to_dataitem().get())).values.tolist()[-5:]}
)

> 2022-12-21 07:42:29,671 [info] invoking function: {'method': 'POST', 'path': 'http://nuclio-testing-notebooks1-dani-proj2-serving.default-tenant.svc.cluster.local:8080/v2/models/my_model/infer'}


{'id': '7aef8dd5-5320-4ccd-88f9-ac561833ad87',
 'model_name': 'my_model',
 'outputs': [2, 2, 2, 2, 2]}

### 4.2 mlrun function

In [22]:
# exporting project2 log_transaction mlrun function
project2.get_function('log_transactions').export('proj2_mlrun_func.yaml')

# Importing project2 log_transaction mlrun function from project1
project1.set_function(name='proj1_imported_mlrun_func', func='../proj2_mlrun_func.yaml')
project1.run_function('proj1_imported_mlrun_func')

> 2022-12-21 07:42:40,605 [info] function spec saved to path: proj2_mlrun_func.yaml
> 2022-12-21 07:42:40,673 [info] starting run log-transactions-log_transactions uid=f1abb05de37849439793d5431a93eb24 DB=http://mlrun-api:8080
> 2022-12-21 07:42:40,902 [info] Job is running in the background, pod: log-transactions-log-transactions-h6cp6
> 2022-12-21 07:42:44,300 [info] dataframe shape : (150, 4)
> 2022-12-21 07:42:44,438 [info] logged model : <class 'xgboost.sklearn.XGBClassifier'>
> 2022-12-21 07:42:44,572 [info] logged dataset (150, 5)
> 2022-12-21 07:42:44,621 [info] To track results use the CLI: {'info_cmd': 'mlrun get run f1abb05de37849439793d5431a93eb24 -p testing-notebooks1-dani', 'logs_cmd': 'mlrun logs f1abb05de37849439793d5431a93eb24 -p testing-notebooks1-dani'}
> 2022-12-21 07:42:44,621 [info] Or click for UI: {'ui_url': 'https://dashboard.default-tenant.app.jmglmvqnganv.iguazio-cd1.com/mlprojects/testing-notebooks1-dani/jobs/monitor/f1abb05de37849439793d5431a93eb24/overview'

project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
testing-notebooks1-dani,...1a93eb24,0,Dec 21 07:42:43,completed,log-transactions-log_transactions,v3io_user=danikind=jobowner=danimlrun/client_version=1.2.1-rc4host=log-transactions-log-transactions-h6cp6,,,,encoded_iris-v3bst_modelmy_projectiris_dataset-v3


> 2022-12-21 07:42:47,224 [info] run executed, status=completed


### 4.3 nuclio function

#### 4.3.1 Importing & Exporting deployed remote function

In [23]:
%%writefile script.py
import mlrun
import time

def handler(context: mlrun.MLClientCtx, event):
    context.logger.info('Going to sleep zZz...')
    time.sleep(5)
    return 'Waking up !'

Writing script.py


In [24]:
# Setting remote function
project1.set_function(func = mlrun.code_to_function(name='proj1_remote_func', kind='remote',image='mlrun/mlrun', filename='script.py', handler='handler'))

# Deploying
project1.deploy_function('proj1-remote-func')
# Exporting project1 undeployed remote function
project1.get_function('proj1-remote-func').export('proj1_remote_func.yaml')

# Importing project2 remote function from project1
project2.set_function(name='proj2_imported_remote_func', func='../proj1_remote_func.yaml')

> 2022-12-21 07:42:47,299 [info] Starting remote function deploy
2022-12-21 07:42:47  (info) Deploying function
2022-12-21 07:42:47  (info) Building
2022-12-21 07:42:47  (info) Staging files and preparing base images
2022-12-21 07:42:47  (info) Building processor image
2022-12-21 07:44:02  (info) Build complete
> 2022-12-21 07:44:22,797 [info] successfully deployed function: {'internal_invocation_urls': ['nuclio-testing-notebooks1-dani-proj1-remote-func.default-tenant.svc.cluster.local:8080'], 'external_invocation_urls': ['testing-notebooks1-dani-proj1-remote-func-testing-note-azymnjtq.default-tenant.app.jmglmvqnganv.iguazio-cd1.com/']}
> 2022-12-21 07:44:22,909 [info] function spec saved to path: proj1_remote_func.yaml


In [25]:
project2.deploy_function('proj2_imported_remote_func')

time.sleep(5)
project2.get_function('proj2_imported_remote_func').invoke('')

> 2022-12-21 07:44:22,931 [info] Starting remote function deploy
2022-12-21 07:44:23  (info) Deploying function
2022-12-21 07:44:23  (info) Building
2022-12-21 07:44:23  (info) Staging files and preparing base images
2022-12-21 07:44:23  (info) Building processor image
2022-12-21 07:45:28  (info) Build complete
2022-12-21 07:46:22  (info) Function deploy complete
> 2022-12-21 07:46:23,286 [info] successfully deployed function: {'internal_invocation_urls': ['nuclio-testing-notebooks2-dani-proj1-remote-func.default-tenant.svc.cluster.local:8080'], 'external_invocation_urls': ['testing-notebooks2-dani-proj1-remote-func-testing-note-lql4if7h.default-tenant.app.jmglmvqnganv.iguazio-cd1.com/']}
> 2022-12-21 07:46:28,478 [info] invoking function: {'method': 'GET', 'path': 'http://nuclio-testing-notebooks2-dani-proj1-remote-func.default-tenant.svc.cluster.local:8080/'}


b'Waking up !'

## 3. Importing/Exporting project

In [26]:
# Creating a new porject
new_project = mlrun.get_or_create_project(name='testing-exported-proj', context=os.path.join(os.getcwd(), 'new-project'))

# Make sure that new_project artifact_path is set to s3

# Setting artifacts
new_project.set_artifact('encoded_iris', artifact=os.path.join(os.getcwd(),'proj2_encoded_iris.yaml'))
new_project.set_artifact('model', artifact=os.path.join(os.getcwd(),'proj1_model.yaml'))
new_project.set_artifact('dataset', artifact=os.path.join(os.getcwd(),'proj2_dataset.yaml'))
# new_project.set_artifact('dataset', artifact=os.path.join(os.getcwd(),'proj1_dir.yaml'))

# Setting functions
new_project.set_function(name='serving', func='../proj1_serving.yaml')
new_project.set_function(name='mlrun_func', func='../proj2_mlrun_func.yaml')
new_project.set_function(name='remote_func', func='../proj1_remote_func.yaml')
# new_project.set_artifact('dir', artifact=os.path.join(os.getcwd(),'proj1_dir.yaml'))

new_project.artifact_path = 's3://testbucket-igz/'

> 2022-12-21 07:46:48,818 [info] Created and saved project testing-exported-proj: {'from_template': None, 'overwrite': False, 'context': '/User/test-notebooks/project_transfer/new-project', 'save': True}
> 2022-12-21 07:46:48,819 [info] created project testing-exported-proj and saved in MLRun DB


In [27]:
AWS_ACCESS_KEY_ID = os.environ.get("AWS_ACCESS_KEY_ID", None)
AWS_SECRET_ACCESS_KEY = os.environ.get("AWS_SECRET_ACCESS_KEY", None)
GOOGLE_APPLICATION_CREDENTIALS = os.environ.get("GOOGLE_APPLICATION_CREDENTIALS", None)
assert AWS_ACCESS_KEY_ID != None and AWS_SECRET_ACCESS_KEY != None and GOOGLE_APPLICATION_CREDENTIALS != None and new_project.artifact_path != None

In [28]:
# exporting the project

# S3 artifact path (e.g. s3://my-bucket/new_project.zip)
new_project.export(filepath=os.path.join(new_project.artifact_path, 'new_project.zip'))

# GCS artifact path (e.g. gs://my-bucket/new_project.zip)
new_project.export(filepath=os.path.join('gs' + new_project.artifact_path[2:], 'new_project.zip'))

# V3IO local path
new_project.export(filepath='/v3io/bigdata/new_project.zip')

In [29]:
# Importing the projects
gs_project = mlrun.load_project(name='testing-gs-project',
                                url=os.path.join(new_project.artifact_path, 'new_project.zip'),
                                context = os.path.join(os.getcwd(), 'gs-project'))

s3_project = mlrun.load_project(name='testing-s3-project',
                                url=os.path.join('s3' + new_project.artifact_path[2:], 'new_project.zip'),
                                context = os.path.join(os.getcwd(), 's3-project'))

v3io_project = mlrun.load_project(name='testing-v3io-project',
                                  url='/v3io/bigdata/new_project.zip',
                                  context=os.path.join(os.getcwd(), 'v3io-project'))

In [30]:
# Testing the imported artifacts

# Artifact
gs_encoded_dataset = pd.DataFrame(json.loads(gs_project.get_artifact('encoded_iris').to_dataitem().get()))
print('gs project artifact\n', gs_encoded_dataset.head(), '\n\n')

# Dataset
s3_dataset = s3_project.get_artifact('dataset').to_dataitem().as_df()
print('s3 project artifact\n',s3_dataset.head())

# Model
v3io_model = pickle.loads(v3io_project.get_artifact('model')._get_file_body())
v3io_model.predict(gs_encoded_dataset[:5])

# Testing imported functions

# Serving function
gs_project.get_function('serving').deploy()
time.sleep(5)
gs_project.get_function('serving').invoke(
    path='/v2/models/my_model/infer', 
    body={'inputs': gs_encoded_dataset.values[-5:].tolist()}
)

# mlrun function
s3_project.run_function('mlrun_func',local=True)

## nuclio function
v3io_project.deploy_function('remote_func')

time.sleep(5)
v3io_project.get_function('remote_func').invoke('')

gs project artifact
      0    1    2    3
0  5.1  3.5  1.4  0.2
1  4.9  3.0  1.4  0.2
2  4.7  3.2  1.3  0.2
3  4.6  3.1  1.5  0.2
4  5.0  3.6  1.4  0.2 


s3 project artifact
    Unnamed: 0    0    1    2    3
0           0  5.1  3.5  1.4  0.2
1           1  4.9  3.0  1.4  0.2
2           2  4.7  3.2  1.3  0.2
3           3  4.6  3.1  1.5  0.2
4           4  5.0  3.6  1.4  0.2
> 2022-12-21 07:47:37,539 [info] Starting remote function deploy
2022-12-21 07:47:37  (info) Deploying function
2022-12-21 07:47:37  (info) Building
2022-12-21 07:47:37  (info) Staging files and preparing base images
2022-12-21 07:47:37  (info) Building processor image
2022-12-21 07:49:43  (info) Build complete
2022-12-21 07:50:23  (info) Function deploy complete
> 2022-12-21 07:50:23,780 [info] successfully deployed function: {'internal_invocation_urls': ['nuclio-testing-gs-project-proj1-serving.default-tenant.svc.cluster.local:8080'], 'external_invocation_urls': ['testing-gs-project-proj1-serving-testing-gs-pr

project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
testing-s3-project,...96ac13de,0,Dec 21 07:50:29,completed,log-transactions-log_transactions,v3io_user=danikind=owner=danihost=jupyter-dani-86ccdd8c48-kztxz,,,,encoded_iris-s3bst_modelmy_projectiris_dataset-s3


> 2022-12-21 07:50:32,418 [info] run executed, status=completed
> 2022-12-21 07:50:32,496 [info] Starting remote function deploy
2022-12-21 07:50:32  (info) Deploying function
2022-12-21 07:50:32  (info) Building
2022-12-21 07:50:32  (info) Staging files and preparing base images
2022-12-21 07:50:32  (info) Building processor image
2022-12-21 07:51:47  (info) Build complete
2022-12-21 07:52:24  (info) Function deploy complete
> 2022-12-21 07:52:24,254 [info] successfully deployed function: {'internal_invocation_urls': ['nuclio-testing-v3io-project-proj1-remote-func.default-tenant.svc.cluster.local:8080'], 'external_invocation_urls': ['testing-v3io-project-proj1-remote-func-testing-v3io-project.default-tenant.app.jmglmvqnganv.iguazio-cd1.com/']}
> 2022-12-21 07:52:29,470 [info] invoking function: {'method': 'GET', 'path': 'http://nuclio-testing-v3io-project-proj1-remote-func.default-tenant.svc.cluster.local:8080/'}


b'Waking up !'